In [1]:
from models.togetherai_model import TogetherAIModel
from services.faiss_engine import FaissEngine
import numpy as np
import pymupdf

client = TogetherAIModel("","togethercomputer/m2-bert-80M-8k-retrieval", "https://api.together.xyz/v1/embeddings")

In [3]:
d = 786

chunks = [
    "The sun was setting over the ocean, casting a warm golden light over the beach.",
    "The smell of freshly brewed coffee filled the air, enticing the sleepy office workers to rise and shine.",
    "In a world where time was currency, the rich lived forever and the poor were left with nothing but memories.",
    "As she walked through the old antique shop, she stumbled upon a mysterious locket that revealed a hidden family secret.",
    "The stars twinkled like diamonds in the night sky, guiding the lost traveler towards a new and exciting destination.",
    "Only the brightest bananas are the best",
    "How do you make banana bread?"
]
np_emb = client.embedding(chunks).embeddings
print(np_emb.shape)

query = [
    "What is the best way to find bananas?",
]
np_query = client.embedding(query).embeddings
np_query.shape

fe = FaissEngine()
fe.add_embeddings(np_emb)
results = fe.search(np_query, topk=2)
print(results)
indexes = results.indices
sent_result= [chunks[i] for i in indexes]
print(sent_result)



(7, 768)
['How do you make banana bread?', 'Only the brightest bananas are the best']
VectorSearchResponse(indices=array([6, 5]), distances=array([20.78393, 22.69541], dtype=float32))


In [4]:
from services.rag_manager import RAGManager
from services.document_engine import DocumentEngine


rag = RAGManager(client, DocumentEngine(500, 10), FaissEngine())
response = rag.process_document('./llm.pdf')


no. of chunks:  6


In [6]:
response = rag.search_similar_chunks("What is the best way to find bananas?", topk=5)
response

['ul launch and scaling of new \nand innovative products. \n• \nProficiency in modern frameworks/languages such as Python, JavaScript, Java, Golang, Scala, \netc. \n• \nExceptional collaboration and communication skills to effectively engage and work with \nstakeholders from diverse disciplines and varying levels of technical expertise. \n• \nAbility to translate complex technical concepts into clear, actionable insights for all \naudiences. \n• \nExperience with AI technologies, particularly in LLMs and mach',
 'eams, including product managers and domain experts, to \ndiscover, build, and iterate on innovative features and products. \n• \nEnsure the scalability, security, and performance of our software systems. \n• \nRapidly develop and iterate on proofs-of-concept to test new ideas and hypotheses, \nefficiently validating and evolving our software solutions. \n• \nStay updated with emerging trends and advancements in AI, LLMs, and software \ndevelopment to continually enhance our p

# TinyDB

In [1]:
from tinydb import TinyDB, Query
db = TinyDB('./db/db.json')
User = Query()

In [7]:

db.insert({'name': 'John', 'age': 22})
db.search(User.name == 'John')

[{'name': 'John', 'age': 22}]

In [3]:
db.search((User.name == 'John') & (User.age <= 30))

[{'name': 'John', 'age': 22}]

In [4]:
db.search((User.name == 'John') | (User.name == 'Bob'))

[{'name': 'John', 'age': 22}]

In [5]:
db.search((User.age.map(lambda x: x + x) == 44))

[{'name': 'John', 'age': 22}]

In [17]:
# insert documents

db.insert({'type': 'apple', 'count': 7})
db.insert({'type': 'peach', 'count': 3})


# Iterate over stored documents

for item in db:
    print(item)


# Query documents

Fruit = Query()
db.search(Fruit.type == 'peach')
# [{'count': 3, 'type': 'peach'}]
db.search(Fruit.count > 5)
# [{'count': 7, 'type': 'apple'}]

# Remove documents
db.remove(Fruit.count < 5)

In [ ]:
# drop db
db.truncate()

# Tables

In [11]:
table = db.table('users')
table.insert({'user_id': 1, "username": "Aemilius" })
table.all()

[{'user_id': 1, 'username': 'Aemilius'}]

In [ ]:
el = db.get(User.username == 'Aemilius')
el.doc_id

In [ ]:
table = db.table('users')
table.insert({'user_id': 1, "username": "Aemilius" })
table.all()

In [5]:
table = db.table('prompt_templates')
table.insert({"user": "Emile", "name": "Summarize", "text": "Summarize the following text {}"})
table.all()

[{'user_id': 1, 'username': 'Aemilius'},
 {'name': 'Summarize', 'text': 'Summarize the following text \\{\\}'},
 {'name': 'Summarize', 'text': 'Summarize the following text {}'},
 {'user': 'Emile',
  'name': 'Summarize',
  'text': 'Summarize the following text {}'}]

<!-- >>> from tinydb.storages import JSONStorage
>>> from tinydb.middlewares import CachingMiddleware
>>> db = TinyDB('/path/to/db.json', storage=CachingMiddleware(JSONStorage)) -->

In [ ]:
table = db.table('prompt_templates')
prompt_template = Query()
table.update({"name": "Summarize", "text": "Summarize the following bva {}"}, prompt_template.user == '', prompt_template.name == 'Summarize' )
table.all()

In [18]:
# initilize DB
def load_db(db_path: str = './db/db.json'):
    db = TinyDB(db_path)
    return db

def initialize_db(db_path: str = './db/db.json'):
    db = TinyDB(db_path)
    table = db.table('users')
    table.insert({"username": "Aemilius"})
    return True

[{'user_id': 1, 'username': 'Aemilius'}, {'username': 'Aemilius'}]